# Measurement Grouping

- We check how many qubit wise commuting groups are there for respective molecules.
- While circuit depth will be lowered using sub-Hamiltonian, qubit count reduction seems not sufficient

In [22]:
from utility import *


def qubits_in_H(H):
    return len(set(list(map(lambda t: t[0], sum(H.terms.keys(), ())))))

## LiH

In [25]:
lih, _ = get_qubit_hamiltonian(mol='lih', geometry=1, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(lih)
print('Number of fragments: {}'.format(len(qwc_list)))
print(f'Qubits required for original hamiltonian: {qubits_in_H(lih)}')

Number of fragments: 154
Qubits required for original hamiltonian: 12


In [26]:
comm_groups = get_commuting_group(lih)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
for i, H in enumerate(comm_groups.values()):
    print(f'Qubits for commuting group {i}: ', qubits_in_H(H))

Number of mutually commuting fragments: 36
Qubits for commuting group 0:  12
Qubits for commuting group 1:  12
Qubits for commuting group 2:  12
Qubits for commuting group 3:  12
Qubits for commuting group 4:  12
Qubits for commuting group 5:  12
Qubits for commuting group 6:  12
Qubits for commuting group 7:  8
Qubits for commuting group 8:  12
Qubits for commuting group 9:  12
Qubits for commuting group 10:  12
Qubits for commuting group 11:  12
Qubits for commuting group 12:  12
Qubits for commuting group 13:  12
Qubits for commuting group 14:  12
Qubits for commuting group 15:  12
Qubits for commuting group 16:  12
Qubits for commuting group 17:  12
Qubits for commuting group 18:  12
Qubits for commuting group 19:  12
Qubits for commuting group 20:  12
Qubits for commuting group 21:  12
Qubits for commuting group 22:  12
Qubits for commuting group 23:  12
Qubits for commuting group 24:  11
Qubits for commuting group 25:  12
Qubits for commuting group 26:  12
Qubits for commuting gr

To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

## H$_4$


In [27]:
h4, _ = get_qubit_hamiltonian(mol='h4', geometry=90, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(h4)
print('Number of fragments: {}'.format(len(qwc_list)))
print(f'Qubits required for original hamiltonian: {qubits_in_H(h4)}')


comm_groups = get_commuting_group(h4)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
for i, H in enumerate(comm_groups.values()):
    print(f'Qubits for commuting group {i}: ', qubits_in_H(H))

Number of fragments: 33
Qubits required for original hamiltonian: 8
Number of mutually commuting fragments: 4
Qubits for commuting group 0:  8
Qubits for commuting group 1:  8
Qubits for commuting group 2:  8
Qubits for commuting group 3:  8


## N$_2$


In [28]:
n2, _ = get_qubit_hamiltonian(mol='n2', geometry=1.5, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(n2)
print('Number of fragments: {}'.format(len(qwc_list)))
print(f'Qubits required for original hamiltonian: {qubits_in_H(n2)}')


comm_groups = get_commuting_group(n2)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
for i, H in enumerate(comm_groups.values()):
    print(f'Qubits for commuting group {i}: ', qubits_in_H(H))

Number of fragments: 1070
Qubits required for original hamiltonian: 20
Number of mutually commuting fragments: 91
Qubits for commuting group 0:  20
Qubits for commuting group 1:  20
Qubits for commuting group 2:  20
Qubits for commuting group 3:  20
Qubits for commuting group 4:  20
Qubits for commuting group 5:  20
Qubits for commuting group 6:  20
Qubits for commuting group 7:  20
Qubits for commuting group 8:  20
Qubits for commuting group 9:  20
Qubits for commuting group 10:  20
Qubits for commuting group 11:  20
Qubits for commuting group 12:  20
Qubits for commuting group 13:  20
Qubits for commuting group 14:  20
Qubits for commuting group 15:  20
Qubits for commuting group 16:  20
Qubits for commuting group 17:  20
Qubits for commuting group 18:  20
Qubits for commuting group 19:  20
Qubits for commuting group 20:  20
Qubits for commuting group 21:  20
Qubits for commuting group 22:  20
Qubits for commuting group 23:  20
Qubits for commuting group 24:  20
Qubits for commuting 

## NH$_3$

In [29]:
nh3, _ = get_qubit_hamiltonian(mol='nh3', geometry=1.5, basis='sto-3g', qubit_transf='jw')

qwc_list = get_qwc_group(nh3)
print('Number of fragments: {}'.format(len(qwc_list)))
print(f'Qubits required for original hamiltonian: {qubits_in_H(nh3)}')

comm_groups = get_commuting_group(nh3)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
for i, H in enumerate(comm_groups.values()):
    print(f'Qubits for commuting group {i}: ', qubits_in_H(H))


Number of fragments: 1200
Qubits required for original hamiltonian: 16
Number of mutually commuting fragments: 152
Qubits for commuting group 0:  16
Qubits for commuting group 1:  16
Qubits for commuting group 2:  16
Qubits for commuting group 3:  16
Qubits for commuting group 4:  16
Qubits for commuting group 5:  16
Qubits for commuting group 6:  16
Qubits for commuting group 7:  16
Qubits for commuting group 8:  16
Qubits for commuting group 9:  16
Qubits for commuting group 10:  16
Qubits for commuting group 11:  16
Qubits for commuting group 12:  16
Qubits for commuting group 13:  16
Qubits for commuting group 14:  16
Qubits for commuting group 15:  16
Qubits for commuting group 16:  16
Qubits for commuting group 17:  16
Qubits for commuting group 18:  16
Qubits for commuting group 19:  16
Qubits for commuting group 20:  16
Qubits for commuting group 21:  16
Qubits for commuting group 22:  16
Qubits for commuting group 23:  16
Qubits for commuting group 24:  16
Qubits for commuting